## 1. Load voxel model

In [12]:
from setup.voxel_setup import setup_voxel_scene
from common.plot import Plotter
from common.figure import *
from simulation.simulator import get_irrad_loc_dir, compute_ior_gradient

import taichi as ti
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2
# May comment it because the compatibility of this extension is not good
%matplotlib widget 

# debug=True to check boundary access
ti.init(arch=ti.gpu)

######## Prefix for scattering only rendering or reflection only rendering ########
PREFIX = "scatter-" #"reflect-"  #""  # "" #  

SCENE_CFG = {    

    "Name": "bunny", # Remember to change the ray marching's THRESHOLD to geometry's
    "Cam Pos": [(4, -1, -1.5), (-3, 1, 2), (-1, 0, 2.1)],

    # "Name": "bunny",  # Remember to change the ray marching's THRESHOLD to bunny's
    # "Cam Pos": [(4, -1, -1.5), (-3, 2.5, 2), (0, 1, 2.1)],


    "Screen Res": (1280, 960),
    "HDR Res": (2000, 1000),
    "HDR Name": "Light_wooden_frame_room_2k.hdr",
    
    "Num XYZ": (128, 128, 128),
    'Floor Ratio': -0.95,

    "Sampler Num": 8,

    "Load Save": True,

    "Save Fig": False,
}

PROC_CFG = {
    "Gauss Sigma": 4.0,
    "Gauss Radius": 2,

    "Grad Threshold": 0.0,
}

plotter = Plotter(SCENE_CFG)
scene = setup_voxel_scene(SCENE_CFG)

scene.apply_filter(PROC_CFG)
scene.gradient = compute_ior_gradient(scene.ior)
# plotter.plot_wavefront(scene.ior, None, None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[Taichi] Starting on arch=cuda
Floor Ratio: -0.95 , Floor Height: 3
Loaded Voxel shape: (128, 128, 128)  from: ./assets/bun_zipper_res3.ply
Number of filled voxels: 237786


In [ ]:
# Average frames per second:

# geometry scene:
# index 0: 27.17 fps
# index 1: 28.96 fps
# index 2: 28.93 fps


# bunny scene:
# index 0: 34.35 fps
# index 1: 35.31 fps
# index 2: 38.84 fps

# 1% low frames per second:

# geometry scene:
# index 0: 24.34 fps
# index 1: 24.78 fps
# index 2: 25.02 fps

# bunny scene:
# index 0: 31.58 fps
# index 1: 33.35 fps
# index 2: 35.81 fps

In [15]:
import open3d as o3d

def visualize_voxels(scene_ior):
    # 假设scene_ior是一个三维numpy数组
    x, y, z = scene_ior.shape
    
    # 创建一个点云来表示非零体素
    points = np.argwhere(scene_ior > 0).astype(np.float32)
    colors = np.array([scene_ior[tuple(point)] for point in points.astype(int)])
    
    # 归一化颜色值到0-1范围
    colors = (colors - np.min(colors)) / (np.max(colors) - np.min(colors))
    colors = np.column_stack((colors, colors, colors))  # 创建RGB颜色

    # 创建Open3D点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # 创建一个可视化窗口
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # 添加点云到可视化窗口
    vis.add_geometry(pcd)

    # 设置一些渲染选项
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])  # 灰色背景
    opt.point_size = 3.0  # 增加点的大小以更好地模拟体素

    # 设置相机视角
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    ctr.set_front([0.5, -0.5, 0.0])
    ctr.set_lookat([x/2, y/2, z/2])
    ctr.set_up([0, 0, 1])

    # 运行可视化
    vis.run()
    vis.destroy_window()

# 示例使用
scene_ior = scene.ior - 1.0
visualize_voxels(scene_ior)

## 2. Perform light simulation

In [ ]:
########## 3D IOR Gradient testing ##########
# plotter.plot_gradient(scene.gradient)
# y_max = 20
# for x in range(77, 90, 2):
#     grad_length_list = []
#     ior_list = []
#     for vec in scene.gradient[x, y_max, :]:
#         cur_len = np.linalg.norm(vec) * 10
#         grad_length_list.append(cur_len)
#     for vec in scene.ior[x, y_max, :]:
#         ior_list.append(vec)
#     plt.plot(grad_length_list, label="length(grad)")
#     plt.plot(ior_list, label="IOR")
#     plt.legend()
#     plt.title("Length(Gradient). x = {}".format(x))
#     plt.show()
# ndimage gaussian

########## 2D IOR Projection ##########
from scipy.ndimage import gaussian_filter

input_3d_ior = scene.ior
ior_2d_projection = np.max(input_3d_ior, axis=2)[:, ::-1].T
ior_2d_projection = gaussian_filter(ior_2d_projection, sigma=4.0, radius=2)
print(ior_2d_projection.shape)
# visualize the 2D projection of IOR withou plotter
plt.figure(figsize=(10, 10))
plt.imshow(ior_2d_projection, cmap='Blues', vmin=1.0, vmax=1.5)
plt.colorbar()
plt.title("2D Projection of IOR")
plt.show()
plt.figure(figsize=(10, 10))
plt.imshow(scene.ior[:, 28, :], cmap='Blues', vmin=1.0, vmax=1.5)
plt.colorbar()
plt.title("2D Projection of IOR")
plt.show()

# scene_name = SCENE_CFG["Name"]
# np_save_path = os.path.join(os.getcwd(), "wavefront2d", scene_name + "-2d_ior.npy")
# np.save(convex_save_path, ior_2d_projection)

In [ ]:
scene.irradiance, scene.local_diretion = get_irrad_loc_dir(scene, SCENE_CFG, plotter=plotter,
                                                           num_show_images=2)
# plotter.plot_irradiance_grid(scene.irradiance)
# plotter.plot_irradiance_slices(scene.irradiance, "np-irrad", num_slices=4, z_start=30, z_end=100)
# plotter.plot_local_direction_grid_slices(scene.local_diretion[:, floor_height:, :], num_slices=4, z_start=30, z_end=100)

## 3. Ray marching render

In [ ]:
scene.rt_render(free_mode=False)
# save_offline_render(scene, SCENE_CFG, filename=PREFIX+"Origin", to_plot=True)

## 4. Different data structures to store or fit irradiance

In [ ]:
from data.siren import SirenFitter, siren_post_process
from data.mlp import MLPFitter, mlp_post_process
from data.octree import Octree, octree_post_process

### 4.1 SIREN to fit irradiance

In [ ]:
siren_fitter= SirenFitter(scene.irradiance, SCENE_CFG,
                     hidden_features=256, hidden_layers=3, omega=24)
siren_fitter.fit(total_epochs=30, batch_size=20000, lr=5e-4) 

In [ ]:
siren_res = siren_fitter.infer()
plotter.plot_irradiance_slices(siren_res, "siren-irrad", threshold=3, num_slices=4, z_start=30, z_end=100)
siren_res.shape

In [ ]:
# corrected_siren_res = siren_post_process(siren_res, gamma=None)
# plotter.plot_irradiance_slices(corrected_siren_res, "corrected-siren-irrad", threshold=3, num_slices=4, z_start=30, z_end=100)
# assert corrected_siren_res.shape == scene.irradiance.shape, "The shape of the corrected siren result should be the same as the original irradiance grid"

In [ ]:
scene.irradiance = siren_res

# scene.rt_render(False)
save_offline_render(scene, SCENE_CFG, filename=PREFIX+"Siren", to_plot=True)

### 4.2 MLP to fit irradiance

In [ ]:
mlp = MLPFitter(scene.irradiance, SCENE_CFG, num_epoches=1000)

In [ ]:
# Visualize the predicted irradiance field
mlp_predicted_irradiance = mlp.predict(pad=True)
plotter.plot_irradiance_slices(mlp_predicted_irradiance, "mlp-irrad", threshold=3, 
                               num_slices=4, z_start=30, z_end=100)

In [ ]:
# corrected_mlp_res = mlp_post_process(mlp_predicted_irradiance, None)
# plotter.plot_irradiance_slices(corrected_mlp_res, "corrected-mlp-irrad", threshold=3, num_slices=4, z_start=30, z_end=100)
# assert corrected_mlp_res.shape == scene.irradiance.shape, "The shape of the corrected mlp result should be the same as the original irradiance grid"

In [ ]:
scene.irradiance = np.clip(mlp_predicted_irradiance, 0, 255)
# scene.rt_render(False)

save_offline_render(scene, SCENE_CFG, filename=PREFIX+"MLP", to_plot=True)

### 4.3 Octree to store irradiance

In [ ]:
octree = Octree(threshold=12)
octree.construct(scene.irradiance)
print(f"Number of nodes: {len(octree)}")
print(f"Octree Memory usage: {octree.__sizeof__()} bytes")
print(f"In comparison, NumPy Storage Usage: {scene.irradiance.nbytes} bytes")
octree.visualize(plotter, "octree-irrad", num_slices=4, z_start=30, z_end=100)

In [ ]:

octree_res = octree.init_empty_grid()
octree.fill_grid(octree.root, octree_res, 0, 0, 0, octree.grid_size)
# plotter.plot_irradiance_slices(octree_res, threshold=3, num_slices=4, z_start=30, z_end=100)
# corrected_octree_res = octree_post_process(octree_res)
# plotter.plot_irradiance_slices(corrected_octree_res, threshold=3, num_slices=4, z_start=30, z_end=100)
# assert corrected_octree_res.shape == scene.irradiance.shape, "The shape of the corrected octree result should be the same as the original irradiance grid"

scene.irradiance = np.clip(octree_res, 0, 255)

# scene.rt_render(False)

save_offline_render(scene, SCENE_CFG, filename=PREFIX+"Octree", to_plot=True)

# x, y, z = 60, 60, 60
# value = octree.query(x, y, z)
# value